In [1]:
from sklearn import datasets
from sklearn.semi_supervised import LabelSpreading

label_prop_model = LabelSpreading()

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [3]:
import json

In [4]:
with open('allsongs.json') as jsonfile:
    data = json.load(jsonfile)

In [5]:
len(data)

3513

In [6]:
def preprocess(item):
    text=""
    for sentence in item:
        sentence = sentence.replace("\t"," ")
        #print(sentence)
        sentence = sentence.replace("\n"," ")
        sentence = sentence.replace("\r"," ")
        sentence = sentence.replace("  "," ")
        sentence = sentence.replace("  "," ")
        sentence = sentence.replace("  "," ")
        sentence = sentence.replace("  "," ")
        #print(sentence)
        text = text+sentence
        
        #print(text)
    text = text.lower()
    text=text.strip()
    return text

In [7]:
T = {}
T['love'] = ['love','romance','romantic','love song','soulmate','relationship','relationships','marriage','crush','love triangle','sweet','young love','true love', 'need'
            ,'young'
            ,'girl'
            ,'care'
            ,'taylor swift'
            ,'heartbeat',
           'understanding',
           'justin',
           'fall out boy']
T['sad']= ['lonely','depression','sad','loneliness','sadness','dark','broken','alone','emo','darkness','isolation','cry','homesick','melancholia','loss','melancholy','memories','regret','nostalgia','homesick','longing','radio'
                    
 ,'forgiveness',
 'screamo',
 'keeper pop punk',
 'grey',
 'storm',
 'light',
 'jazz',
 'irony',
 'inspired by a film',
 'sad song',
 'post-punk', 'melancholic'
                    ]
#T['sad'] = ['lonely','depression','sad','loneliness','sadness','dark','broken','alone','emo','darkness','isolation','cry','homesick']
T['breakup'] = ['breakup','break up','break-up','heartbreak','cheating','betrayal','divorce','infidelity','disillusionment','betrayal','disappointment','guilt','heartache','jealousy','leaving','unrequited love','discontentment', 'mimicking birds', 'prog', 'rain']
#T['nostalgia'] = ['melancholia','loss','melancholy','memories','regret','nostalgia','homesick','longing','radio']
T['death'] = ['suicidal','suicide','death','war','murder','abuse','pain','evil','funeral','drugs','addiction','alcoholism','heroin','bullying','satan','domestic abuse','cancer','cocaine','weed','dying','paranoia',
 'hurt',
 'devil',
 'insanity',
 'shoegaze',
 'foster the people',
 'death metal',
 'hell',
'mother','afterlife'
             ]
T['sex'] = ['sex','rape','sexy','oral sex','masturbation','sexuality','prostitution','lust','virginity','desire','voyeurism','bondage',
 'sexual relationship',
 'lack of romance',
 'domination',
 'one night stand']
T['happy'] = ['happysongs','happy','happiness','beauty','comedy','inspiration','dream','empathy','life','christmas','holiday','hope','faith','friendship','beautiful','amazing','freedom','peace','family','friends','awesome','perseverance','funny',
             'childhood experience'
             ,'challenges',
             'fashion']


In [53]:
M={}
for tag in T:
    for it in T[tag]:
        M[it]=tag

In [54]:
N ={}
N = dict( [ (it[1],it[0]) for it in enumerate(T.keys()) ] )
N

{'breakup': 0, 'death': 1, 'happy': 2, 'love': 4, 'sad': 3, 'sex': 5}

In [55]:
F={}
for it in N:
    F[N[it]]=it 

In [56]:
D={}
#parte de tags
cntSong=0
for item in data:
    sal=preprocess(item['lyric'])
    if(len(sal)<120):
        continue
    ok=0
    
    #print(sal)
    for tag in item['tags']:
        tag=tag.lower()
        if tag in M:
            ok=1
        #print(tag)
        if not tag in D:
            D[tag]=[]
        D[tag].append( sal )
    cntSong+=ok
    #break
print(cntSong)

2619


In [62]:
salida = json.dump(T, open('Tags.json','w'))

In [ ]:
salida

In [34]:
pocos=[]
for tag in D:
    if len(D[tag])==1:
        pocos.append(tag)
#for tag in pocos:
#    D.pop(tag, None)
    
print(len(pocos))

3776


In [35]:
len(D)

4494

In [63]:
label=[]
textos=[]

for tag in D:
    for s in D[tag]:
        label.append(tag)
        textos.append(s)

In [64]:
nlabel=[ (N[M[i]] if (i in M) else -1) for i in label ]

In [65]:
[nlabel.count(i) for i in range(-1,6)] 

[3163, 209, 476, 2439, 379, 531, 171]

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [42]:
 vectorizer = TfidfVectorizer(ngram_range=(1,4),max_features=1000, stop_words='english',min_df=0.005  )

In [43]:
tfidf=vectorizer.fit(textos)

In [44]:
for t in tfidf.vocabulary_:
    if len(t.split())>1:
        print(t)

oh oh
say goodbye
whoa oh
say say
baby baby
don let
know ve
long long
run away
don like
don say
make feel
la la la la
cause know
don think
ll love
know don
know gonna
ah ah
yeah yeah
ain got
wanna know
oh love
ha ha
baby don
don mind
love love love
feels like
set free
time time
want know
love just
want want
just want
just little
ll tell
feel like
oh oh oh
oh don
time know
ooh ooh ooh
don don
little bit
love oh
ll know
don stop
don try
long time
don need
come home
ll make
know know
just like
oh yeah
don know
ve got
don feel
let let
ll ll
hey hey hey hey
love like
know want
know time
just let
just don
don want
know ll
ll come
la la
know just
know don know
don worry
hey hey hey
gone gone
die die
ve come
hold hold
won let
run run
long way
cause don
let know
oh oh oh oh
know love
don care
close eyes
walk away
know got
oh lord
ve seen
say don
oh baby
time ll
need love
know know know
right right
far away
don wanna
know oh
ooh ooh
yeah yeah yeah
love don
yeah yeah yeah yeah
ooh ooh ooh ooh
wai

In [45]:
salida=tfidf.transform(textos)

In [46]:
X_total=salida.toarray()

In [47]:
label_prop_model = LabelSpreading(kernel='rbf', max_iter=5000, n_jobs=100)
label_prop_model.fit(X_total, nlabel)

LabelSpreading(alpha=0.2, gamma=20, kernel='rbf', max_iter=5000, n_jobs=100,
        n_neighbors=7, tol=0.001)

In [48]:
label_prop_model.n_iter_

4999

In [49]:
labN= label_prop_model.label_distributions_

In [50]:
SS={}
for index,tag in enumerate(label):
    if not tag in M: 
        if not tag in SS:
            SS[tag]=[]
            for i in labN[index]:
                SS[tag].append(0)
        SS[tag] = np.sum([ SS[tag],labN[index] ], axis=0)
        #pos=0        
        #for i in labN[index]:
        #    SS[tag][pos] += labN[index][pos]
        #if max(labN[index])>0.5:
        #    act=F[np.argmax(labN[index])]
        #else:
        #    act="otro"
        #SS[tag].append( act )

In [51]:
for it in SS:
    SS[it] = SS[it]/np.sum(SS[it])

In [52]:
cambios=0
for it in SS:
    pos = np.argmax(SS[it])
    if SS[it][pos]<0.99 and SS[it][pos]>0.75 :
        cambios+=1
        T[F[pos]].append(it)
        
print(cambios)

1324
